In [1]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path



In [2]:
%%script false --no-raise-error
#get files from desktop
people = pd.read_csv('/Users/marlenedahle/Desktop/Data Master crunchbase/people.csv')
organizations = pd.read_csv('/Users/marlenedahle/Desktop/Data Master crunchbase/organizations.csv')
acquisitions = pd.read_csv('/Users/marlenedahle/Desktop/Data Master crunchbase/acquisitions.csv')
degrees = pd.read_csv('/Users/marlenedahle/Desktop/Data Master crunchbase/degrees.csv')
#create empty database
Path('second_round_all.db').touch()
#Connect to database 
conn = sqlite3.connect('second_round_all.db')
c = conn.cursor()
#%%script false --no-raise-error

#ORGANIZATIONS create table  
c.execute('''CREATE TABLE organizations (company_name text, 
    roles text, permalink text, domain text, homepage_url text, country_code text,
    state_code text, region text, city text, address text, 
    status text, short_description text, category_list text, category_group_list text,
    funding_rounds int, funding_total_usd float, founded_on date, last_funding_on date,
    closed_on date, employee_count text, email text, phone text, facebook_url text,
    linkedin_url text, cb_url text, logo_url text, twitter_url text, aliases text,
    uuid text PRIMARY KEY, created_at datetime, updated_at datetime, primary_role text,
    type text)''')

#WRITE TO SQL 
#c.execute('''DROP TABLE organizations''')
organizations.to_sql('organizations', conn, if_exists='append', index = False) # write to sqlite table

#PEOPLE
c.execute('''CREATE TABLE people (first_name TEXT, last_name TEXT, country_code int, 
    state_code int, 
    city text, cb_url text, logo_url text, twitter_url TEXT, facebook_url TEXT, 
    linkedin_url TEXT, primary_affiliation_organization TEXT, primary_affiliation_title,
    primary_organization_uuid TEXT, gender TEXT, uuid TEXT,
    created_at datetime, updated_at datetime,
    PRIMARY KEY (uuid), 
    FOREIGN KEY (primary_organization_uuid)
        REFERENCES organizations (uuid) )''')
# write the data to a sqlite table
people.to_sql('people', conn, if_exists='append', index = False)

#ACQUISITIONS 
c.execute('''CREATE TABLE acquisitions (acquiree_name text, acquiree_country_code text,
    state_code text, acquiree_region text, acquiree_city text, acquirer_name text,
    acquirer_country_code text, acquirer_state_code text, acquirer_region text, 
    acquirer_city text, acquisition_type text, acquired_on date, price_usd float,
    price float, price_currency_code text, acquiree_cb_url text, 
    acquirer_cb_url text, acquirer_uuid text, acquiree_uuid text,
    acquisition_uuid text, created_at datetime, updated_at datetime,
    PRIMARY KEY (acquisition_uuid), 
    FOREIGN KEY (acquirer_uuid)
        REFERENCES organizations (uuid) 
    FOREIGN KEY (acquiree_uuid) 
        REFERENCES organizations (uuid) )''')
acquisitions.to_sql('acquisitions', conn, if_exists='append', index = False) 

#DEGREES TABLE 
c.execute('''CREATE TABLE degrees (degree_uuid text, institution_uuid text, 
    person_uuid text, degree_type text, subject text, started_on date, completed_on date,
    is_completed text, created_at datetime, updated_at datetime,
    PRIMARY KEY (degree_uuid),
    FOREIGN KEY (person_uuid)
        REFERENCES people (uuid) )''')
degrees.to_sql('degrees', conn, if_exists = 'append', index = False)

### START BY FILTERING ORGANIZATIONS on time and type of company 
##Start_ups
c.execute("""CREATE TABLE startups_table AS
    SELECT company_name, uuid, category_list, category_group_list, region,
    city, founded_on, primary_role, status, homepage_url
    FROM organizations
    WHERE founded_on BETWEEN '1995-01-01' 
    AND '2015-12-31' 
    AND primary_role = 'company' """)
#JOIN all tables to one dataframe 
df_all_data = pd.read_sql('''SELECT 
    O.uuid as ORG_org_uuid, O.company_name, O.category_list, 
    O.category_group_list, O.region, O.city, O.founded_on, O.status, O.homepage_url,
    P.gender, P.primary_organization_uuid AS PER_org_uuid, P.uuid as PER_pers_uuid,
    D.started_on, D.completed_on, D.is_completed, D.person_uuid as DEG_person_uuid 
    FROM startups_table as O
    LEFT JOIN people as P ON P.primary_organization_uuid = O.uuid 
    LEFT JOIN degrees as D ON D.person_uuid = P.uuid
    AND PER_org_uuid != 'None'
    ''', conn)

#%%script false --no-raise-error

#ORGANIZATIONS create table  
c.execute('''CREATE TABLE organizations (company_name text, 
    roles text, permalink text, domain text, homepage_url text, country_code text,
    state_code text, region text, city text, address text, 
    status text, short_description text, category_list text, category_group_list text,
    funding_rounds int, funding_total_usd float, founded_on date, last_funding_on date,
    closed_on date, employee_count text, email text, phone text, facebook_url text,
    linkedin_url text, cb_url text, logo_url text, twitter_url text, aliases text,
    uuid text PRIMARY KEY, created_at datetime, updated_at datetime, primary_role text,
    type text)''')

#WRITE TO SQL 
#c.execute('''DROP TABLE organizations''')
organizations.to_sql('organizations', conn, if_exists='append', index = False) # write to sqlite table

#PEOPLE
c.execute('''CREATE TABLE people (first_name TEXT, last_name TEXT, country_code int, 
    state_code int, 
    city text, cb_url text, logo_url text, twitter_url TEXT, facebook_url TEXT, 
    linkedin_url TEXT, primary_affiliation_organization TEXT, primary_affiliation_title,
    primary_organization_uuid TEXT, gender TEXT, uuid TEXT,
    created_at datetime, updated_at datetime,
    PRIMARY KEY (uuid), 
    FOREIGN KEY (primary_organization_uuid)
        REFERENCES organizations (uuid) )''')
# write the data to a sqlite table
people.to_sql('people', conn, if_exists='append', index = False)

#ACQUISITIONS 
c.execute('''CREATE TABLE acquisitions (acquiree_name text, acquiree_country_code text,
    state_code text, acquiree_region text, acquiree_city text, acquirer_name text,
    acquirer_country_code text, acquirer_state_code text, acquirer_region text, 
    acquirer_city text, acquisition_type text, acquired_on date, price_usd float,
    price float, price_currency_code text, acquiree_cb_url text, 
    acquirer_cb_url text, acquirer_uuid text, acquiree_uuid text,
    acquisition_uuid text, created_at datetime, updated_at datetime,
    PRIMARY KEY (acquisition_uuid), 
    FOREIGN KEY (acquirer_uuid)
        REFERENCES organizations (uuid) 
    FOREIGN KEY (acquiree_uuid) 
        REFERENCES organizations (uuid) )''')
acquisitions.to_sql('acquisitions', conn, if_exists='append', index = False) 

#DEGREES TABLE 
c.execute('''CREATE TABLE degrees (degree_uuid text, institution_uuid text, 
    person_uuid text, degree_type text, subject text, started_on date, completed_on date,
    is_completed text, created_at datetime, updated_at datetime,
    PRIMARY KEY (degree_uuid),
    FOREIGN KEY (person_uuid)
        REFERENCES people (uuid) )''')
degrees.to_sql('degrees', conn, if_exists = 'append', index = False)

### START BY FILTERING ORGANIZATIONS on time and type of company 
##Start_ups
c.execute("""CREATE TABLE startups_table AS
    SELECT company_name, uuid, category_list, category_group_list, region,
    city, founded_on, primary_role, status, homepage_url
    FROM organizations
    WHERE founded_on BETWEEN '1995-01-01' 
    AND '2015-12-31' 
    AND primary_role = 'company' """)
#JOIN all tables to one dataframe 
df_all_data = pd.read_sql('''SELECT 
    O.uuid as ORG_org_uuid, O.company_name, O.category_list, 
    O.category_group_list, O.region, O.city, O.founded_on, O.status, O.homepage_url,
    P.gender, P.primary_organization_uuid AS PER_org_uuid, P.uuid as PER_pers_uuid,
    D.started_on, D.completed_on, D.is_completed, D.person_uuid as DEG_person_uuid 
    FROM startups_table as O
    LEFT JOIN people as P ON P.primary_organization_uuid = O.uuid 
    LEFT JOIN degrees as D ON D.person_uuid = P.uuid
    AND PER_org_uuid != 'None'
    ''', conn)

In [3]:
df_all_data = pd.read_csv('all_data_1995_2015_prim_comp.csv') #320.915 total rows/companies 

In [4]:
all_data_untouched = df_all_data.copy()
all_data_untouched.info() #582.108 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582108 entries, 0 to 582107
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Unnamed: 0           582108 non-null  int64 
 1   ORG_org_uuid         582108 non-null  object
 2   company_name         582107 non-null  object
 3   category_list        508107 non-null  object
 4   category_group_list  498016 non-null  object
 5   region               426182 non-null  object
 6   city                 502255 non-null  object
 7   founded_on           582108 non-null  object
 8   status               582108 non-null  object
 9   homepage_url         567266 non-null  object
 10  gender               399341 non-null  object
 11  PER_org_uuid         399654 non-null  object
 12  PER_pers_uuid        399654 non-null  object
 13  started_on           40618 non-null   object
 14  completed_on         68396 non-null   object
 15  is_completed         135627 non-nu

In [5]:
#Get right datatype in dates 
df_all_data['started_on'] = pd.to_datetime(df_all_data['started_on'], format='%Y-%m-%d', errors = 'coerce')
df_all_data['completed_on'] = pd.to_datetime(df_all_data['completed_on'], format='%Y-%m-%d', errors = 'coerce')
df_all_data['founded_on'] = pd.to_datetime(df_all_data['founded_on'], format='%Y-%m-%d', errors = 'coerce')
#create variable education time
df_all_data['education_time'] = (df_all_data.completed_on - df_all_data.started_on)
def diff_days(row):
  try:
    return np.busday_count(row['completed_on'].date(), row['founded_on'].date())
  except ValueError:
    return np.NaN

df_all_data['work_experience'] = df_all_data.apply(lambda x: diff_days(x),axis=1)
#test_time = df_all_data.loc[df_all_data['work_experience'].isnull() == False] 
#test_time
#Count whether a person has multiple degrees 
multiple_degrees = df_all_data['DEG_person_uuid'].value_counts()>1
multiple_degrees = multiple_degrees.to_frame('multiple_degrees')
multiple_degrees.index.name = 'person_uuid'
multiple_degrees.reset_index(level=0, inplace=True)
df_all_data = pd.merge(df_all_data, multiple_degrees,  how='left', left_on=['DEG_person_uuid'], right_on = ['person_uuid'])
#test_mult = df_all_data.loc[df_all_data['multiple_degrees'].isnull() == False] 

##How many start-ups are succesful in each city and region, as a percentage? 
df_unique_company = df_all_data.drop_duplicates(subset = ['ORG_org_uuid'])
df_unique_company = df_unique_company[['ORG_org_uuid', 'city', 'status']]
df_unique_company = df_unique_company.replace('None', np.nan, regex=True)
df_unique_company = df_unique_company.dropna(subset=['city'])
#group = city.groupby(['city']).sum()

city_operating = df_unique_company.loc[df_unique_company['status'] == 'operating']
city_acquired = df_unique_company.loc[df_unique_company['status'] == 'acquired']
city_ipo = df_unique_company.loc[df_unique_company['status'] == 'ipo']
city_closed = df_unique_company.loc[df_unique_company['status'] == 'closed']

#How many companies in each city? 
#dictionary
from collections import Counter
#df_unique_company['city'] = df_unique_company['city'].astype(str)
#Counter("".join(df_unique_company["city"]).split()).most_common(10)

letter_counts = Counter(df_unique_company['city'])
all_cities_counts = pd.DataFrame.from_dict(letter_counts, orient='index', columns = ['occurrences'])
all_cities_counts = all_cities_counts.sort_values(by = 'occurrences', ascending = False)
all_cities_counts.index.name = 'city_name'

operating_counter = Counter(city_operating["city"])
operating_counter_df = pd.DataFrame.from_dict(operating_counter, orient='index', columns = ['#operating'])
operating_counter_df = operating_counter_df.sort_values(by = '#operating', ascending = False)
operating_counter_df.index.name = 'city_name'

acquired_counter = Counter(city_acquired["city"])
acquired_counter_df = pd.DataFrame.from_dict(acquired_counter, orient='index', columns = ['#acquired'])
acquired_counter_df = acquired_counter_df.sort_values(by = '#acquired', ascending = False)
acquired_counter_df.index.name = 'city_name'

ipo_counter = Counter(city_ipo["city"])
ipo_counter_df = pd.DataFrame.from_dict(ipo_counter, orient='index', columns = ['#ipo'])
ipo_counter_df = ipo_counter_df.sort_values(by = '#ipo', ascending = False)
ipo_counter_df.index.name = 'city_name'

closed_counter = Counter(city_closed["city"])
closed_counter_df = pd.DataFrame.from_dict(closed_counter, orient='index', columns = ['#closed'])
closed_counter_df = closed_counter_df.sort_values(by = '#closed', ascending = False)
closed_counter_df.index.name = 'city_name'

#Merge all dataframes 
new1 = all_cities_counts.merge(operating_counter_df, on = 'city_name', how = 'left')
new2 = new1.merge(acquired_counter_df, on = 'city_name', how = 'left')
new3 = new2.merge(ipo_counter_df, on = 'city_name', how = 'left')
df_status = new3.merge(closed_counter_df, on = 'city_name', how = 'left')

df_status["operating_prop"] = df_status["#operating"]/df_status["occurrences"]
df_status["acquired_prop"] = df_status["#acquired"]/df_status["occurrences"]
df_status["ipo_prop"] = df_status["#ipo"]/df_status["occurrences"]
df_status["closed_prop"] = df_status["#closed"]/df_status["occurrences"]

#Rank of number of startups 
drop_less_than10 = df_status.drop(df_status[df_status.occurrences<10].index)

df_status['ipo_or_acquired_sum'] = (df_status['#acquired'].fillna(0)+ df_status['#ipo'].fillna(0))
df_status['ipo_or_acquired_prop'] = (df_status['ipo_or_acquired_sum'].fillna(0)/df_status['occurrences'].fillna(0))
df_status['city_success_ranking'] = df_status['ipo_or_acquired_prop'].rank(method = 'max', pct = True)

#Merge all dataframes 
new1 = all_cities_counts.merge(operating_counter_df, on = 'city_name', how = 'left')
new2 = new1.merge(acquired_counter_df, on = 'city_name', how = 'left')
new3 = new2.merge(ipo_counter_df, on = 'city_name', how = 'left')
df_status = new3.merge(closed_counter_df, on = 'city_name', how = 'left')

#calcualte proportion 
df_status["operating_prop"] = df_status["#operating"]/df_status["occurrences"]
df_status["acquired_prop"] = df_status["#acquired"]/df_status["occurrences"]
df_status["ipo_prop"] = df_status["#ipo"]/df_status["occurrences"]
df_status["closed_prop"] = df_status["#closed"]/df_status["occurrences"]

#Rank of number of startups 
drop_less_than10 = df_status.drop(df_status[df_status.occurrences<10].index)

df_status['ipo_or_acquired_sum'] = (df_status['#acquired'].fillna(0)+ df_status['#ipo'].fillna(0))
df_status['ipo_or_acquired_prop'] = (df_status['ipo_or_acquired_sum'].fillna(0)/df_status['occurrences'].fillna(0))
df_status['city_success_ranking'] = df_status['ipo_or_acquired_prop'].rank(method = 'max', pct = True)


In [6]:
#Now replace city names with city success rank. 
df_success_imp = df_status['city_success_ranking']
dict_success = df_success_imp.to_dict()
#df_all_data.replace({"city": dict_success})

#add as column
df_all_data['city_success_ranking'] = df_all_data['city'].map(dict_success)

##How many start-ups are succesful in each city and region, as a percentage? 
df_unique_company_reg = df_all_data.drop_duplicates(subset = ['ORG_org_uuid'])
df_unique_company_reg = df_unique_company_reg[['ORG_org_uuid', 'region', 'status']]
df_unique_company_reg = df_unique_company_reg.replace('None', np.nan, regex=True)
df_unique_company_reg = df_unique_company_reg.dropna(subset=['region'])
counts_reg = Counter(df_unique_company_reg['region'])
all_regions_counts = pd.DataFrame.from_dict(counts_reg, orient='index', columns = ['occurrences'])
all_regions_counts = all_regions_counts.sort_values(by = 'occurrences', ascending = False)
all_regions_counts.index.name = 'region_name'

reg_operating = df_unique_company_reg.loc[df_unique_company_reg['status'] == 'operating']
reg_acquired = df_unique_company_reg.loc[df_unique_company_reg['status'] == 'acquired']
reg_ipo = df_unique_company_reg.loc[df_unique_company_reg['status'] == 'ipo']
reg_closed = df_unique_company_reg.loc[df_unique_company_reg['status'] == 'closed']
acquired_counter_reg = Counter(reg_acquired["region"])
acquired_counter_reg_df = pd.DataFrame.from_dict(acquired_counter_reg, orient='index', columns = ['#acquired'])
acquired_counter_reg_df = acquired_counter_reg_df.sort_values(by = '#acquired', ascending = False)
acquired_counter_reg_df.index.name = 'region_name'
ipo_counter_reg = Counter(reg_ipo["region"])
ipo_counter_reg_df = pd.DataFrame.from_dict(ipo_counter_reg, orient='index', columns = ['#ipo'])
ipo_counter_reg_df = ipo_counter_reg_df.sort_values(by = '#ipo', ascending = False)
ipo_counter_reg_df.index.name = 'region_name'
#Merge all dataframes region acquired, ipo 
reg1 = all_regions_counts.merge(acquired_counter_reg_df, on = 'region_name', how = 'left')
regions_success = reg1.merge(ipo_counter_reg_df, on = 'region_name', how = 'left')
regions_success['ipo_or_acquired_sum'] = (regions_success['#acquired'].fillna(0) + regions_success['#ipo'].fillna(0))
regions_success['ipo_or_acquired_prop'] = (regions_success['ipo_or_acquired_sum'].fillna(0)/regions_success['occurrences'].fillna(0))
regions_success['region_success_ranking'] = regions_success['ipo_or_acquired_prop'].rank(method = 'max', pct = True)

#Now replace region names with region success rank. 
df_success_imp_reg = regions_success['region_success_ranking']
dict_success_reg = df_success_imp_reg.to_dict()

#df_all_data
df_all_data['region_rank'] = df_all_data['region'].map(dict_success_reg)



In [7]:
df_all_data['gender'] = df_all_data.gender.replace({'not_provided': 'unknown', 'non_binary': 'unknown', None : 'unknown', 'male':'MALE'})

#Gender aggregating
grouped_df = df_all_data.groupby("ORG_org_uuid")
grouped_lists = grouped_df["gender"].agg(lambda column: " ".join(column))
gender_list = grouped_lists.reset_index(name="gender")

gender_list['male_founders'] = gender_list.gender.str.count('MALE')
gender_list['female_founders'] = gender_list.gender.str.count('female')
gender_list['unknown_founders'] = gender_list.gender.str.count('unknown')
gender_list['total_num_founders'] = gender_list['male_founders'] + gender_list['female_founders'] + gender_list['unknown_founders']
gender_list

,ORG_org_uuid,gender,male_founders,female_founders,unknown_founders,total_num_founders
0,00001b2d-a4f7-55d5-d69a-17acbac9c17b,unknown,0,0,1,1
1,00002470-bff7-6226-5800-0ca1b3787b6f,MALE MALE MALE MALE,4,0,0,4
2,00003a2c-ad4d-ff61-997b-7b22b2320895,unknown,0,0,1,1
3,000095de-8e2b-82f1-32a7-c222ba3d5682,unknown,0,0,1,1
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,MALE,1,0,0,1
...,...,...,...,...,...,...
320910,ffff25bc-9902-33e8-062e-6d84841d8cf9,unknown,0,0,1,1
320911,ffff3209-5481-022e-cdbf-c38a0a762dc4,unknown,0,0,1,1
320912,ffff8343-97a6-3667-67af-ed3789fa9e14,female,0,1,0,1
320913,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,MALE MALE MALE MALE MALE MALE,6,0,0,6


In [8]:
#Education time average per company
df_all_data['education_time'] = pd.to_numeric(df_all_data['education_time'].dt.days, downcast='integer')
edu_group_list = grouped_df['education_time'].mean()
edu_group_list = edu_group_list.reset_index(name = 'education_time_avg')

#sum
edu_group_list_sum = grouped_df['education_time'].sum()
edu_group_list_sum = edu_group_list_sum.reset_index(name = 'education_time_sum')

In [9]:
#Work experience average per company
work_exp_group_list = grouped_df['work_experience'].mean()
work_exp_group_list = work_exp_group_list.reset_index(name = 'work_experience_avg')

work_exp_group_list_sum = grouped_df['work_experience'].sum()
work_exp_group_list_sum = work_exp_group_list_sum.reset_index(name = 'work_experience_sum')

In [10]:
#completed degree  d
df_all_data['is_completed'] = df_all_data.is_completed.replace({'t': 1, 'f': 0})
#average
completed_degree_list_avg = grouped_df['is_completed'].mean()
completed_degree_list_avg = completed_degree_list_avg.reset_index(name = 'is_completed_avg')

#sum
completed_degree_list_sum = grouped_df['is_completed'].sum()
completed_degree_list_sum = completed_degree_list_sum.reset_index(name = 'is_completed_sum')

In [11]:
#Multiple degrees 
df_all_data['multiple_degrees'] = df_all_data.is_completed.replace({True: 1, False: 0})

#average
multiple_degrees_list_avg = grouped_df['multiple_degrees'].mean()
multiple_degrees_list_avg = multiple_degrees_list_avg.reset_index(name = 'multiple_degrees_average')

#sum
multiple_degrees_list_sum = grouped_df['multiple_degrees'].sum()
multiple_degrees_list_sum = multiple_degrees_list_sum.reset_index(name = 'multiple_degrees_sum')

In [12]:
final_dataframe = df_all_data.drop_duplicates(subset = ['ORG_org_uuid'])
final_dataframe = final_dataframe.sort_values(['ORG_org_uuid'], ascending = True)
final_dataframe

,Unnamed: 0,ORG_org_uuid,company_name,category_list,category_group_list,region,city,founded_on,status,homepage_url,...,started_on,completed_on,is_completed,DEG_person_uuid,education_time,work_experience,person_uuid,multiple_degrees,city_success_ranking,region_rank
523395,523395,00001b2d-a4f7-55d5-d69a-17acbac9c17b,STD Risk Calculator,Biotechnology,"Biotechnology,Science and Engineering",NaN,NaN,2012-03-12,operating,http://www.stdriskcalculator.com,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193685,193685,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,"Developer Tools,Education,Web Development","Education,Software",NaN,Mountain View,2013-01-01,operating,https://www.codementor.io,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.748085,NaN
513734,513734,00003a2c-ad4d-ff61-997b-7b22b2320895,Smart MultiMedia,"Consulting,Data Visualization,Digital Media","Data and Analytics,Design,Information Technolo...",Houston,Houston,2007-01-01,operating,http://www.smartmm.com,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.733041,0.732143
5880,5880,000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,"B2B,E-Commerce",Commerce and Shopping,Hagerstown,Frederick,2001-01-01,operating,NaN,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.732135,0.681548
197774,197774,0000d497-c93a-eea3-eeb0-a943dfb4f71e,AutoOffer,"Automotive,Insurance,Trading Platform","Financial Services,Lending and Investments,Tra...",Copenhagen,Copenhagen,2013-01-01,operating,https://www.autooffer.dk/,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.687143,0.610863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9860,9860,ffff25bc-9902-33e8-062e-6d84841d8cf9,Vivoxid,Health Care,Health Care,Turku,Turku,2001-01-01,closed,http://www.vivoxid.com,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.737359,0.719494
293210,293210,ffff3209-5481-022e-cdbf-c38a0a762dc4,BJJ Jiu-Jitsu,Internet,Internet Services,NaN,NaN,2012-01-01,operating,http://www.jiujitsutoday.com/,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566710,566710,ffff8343-97a6-3667-67af-ed3789fa9e14,Wantist,"E-Commerce,Gift Card,Shopping","Commerce and Shopping,Financial Services","Portland, Oregon",Portland,2010-11-01,operating,http://www.wantist.com,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.683939,0.596726
200671,200671,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,"Enterprise Applications,Finance,Mobile Apps","Apps,Financial Services,Mobile,Software",New York City,New York,2015-01-01,operating,http://www.ernit.com/,...,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.672656,0.519345


In [13]:
dfs = [final_dataframe, multiple_degrees_list_sum, multiple_degrees_list_avg, 
              completed_degree_list_sum, completed_degree_list_avg, work_exp_group_list, 
               work_exp_group_list_sum,
              edu_group_list,edu_group_list_sum, gender_list]
from functools import reduce

df_final_all = reduce(lambda left,right: pd.merge(left,right,on='ORG_org_uuid'), dfs)



In [14]:
df_final_all = df_final_all.drop(columns=['PER_org_uuid', 'PER_pers_uuid',
                                         'started_on','completed_on', 'DEG_person_uuid','person_uuid',
                                         'multiple_degrees', 'gender_y', 'is_completed', 'gender_x','work_experience',
                                         'education_time'])
df_final_all = df_final_all.drop(columns = ['Unnamed: 0'])

In [15]:
#Some more preprocessing on categories 
df_final_all[["category_list1", "category_list2", "category_list3"]]= df_final_all["category_list"].str.split(",", 2, expand = True) 
df_final_all[["category_group_list1", "category_group_list2", "category_group_list3"]]= df_final_all["category_group_list"].str.split(",", 2, expand = True) 

#fill all nan values with 0 
df_final_all = df_final_all.fillna(0)

#CREATE new variable 
df_final_all['mostly_male_founders'] = df_final_all['male_founders'] > df_final_all['female_founders']
df_final_all['mostly_male_founders'] = df_final_all['mostly_male_founders'].astype(int)
df_final_all['mostly_male_founders'].value_counts() #118.311 with mostly male founders, 9196 female

0    204402
1    116513
Name: mostly_male_founders, dtype: int64

In [23]:
##Categories 

df_final_all[["category_list1", "category_list2", "category_list3"]]= df_final_all["category_list"].str.split(",", 2, expand = True) 
df_final_all[["category_group_list1", "category_group_list2", "category_group_list3"]]= df_final_all["category_group_list"].str.split(",", 2, expand = True) 



In [24]:
df_final_all['category_list1'] = df_final_all['category_list1'].astype('category')
df_final_all['category_list2'] = df_final_all['category_list2'].astype('category')
df_final_all['category_list3'] = df_final_all['category_list3'].astype('category')
df_final_all['category_group_list1'] = df_final_all['category_group_list1'].astype('category')
df_final_all['category_group_list2'] = df_final_all['category_group_list2'].astype('category')
df_final_all['category_group_list3'] = df_final_all['category_group_list3'].astype('category')


In [26]:
cat_columns = df_final_all.select_dtypes(['category']).columns
df_final_all[cat_columns] = df_final_all[cat_columns].apply(lambda x: x.cat.codes)



In [27]:
df_final_all

,ORG_org_uuid,company_name,category_list,category_group_list,region,city,founded_on,status,homepage_url,city_success_ranking,...,female_founders,unknown_founders,total_num_founders,category_list1,category_list2,category_list3,category_group_list1,category_group_list2,category_group_list3,mostly_male_founders
0,00001b2d-a4f7-55d5-d69a-17acbac9c17b,STD Risk Calculator,Biotechnology,"Biotechnology,Science and Engineering",0,0,2012-03-12,operating,http://www.stdriskcalculator.com,0.000000,...,0,1,1,65,-1,-1,5,38,-1,0
1,00002470-bff7-6226-5800-0ca1b3787b6f,Codementor,"Developer Tools,Education,Web Development","Education,Software",0,Mountain View,2013-01-01,operating,https://www.codementor.io,0.748085,...,0,0,4,188,205,24820,14,39,-1,1
2,00003a2c-ad4d-ff61-997b-7b22b2320895,Smart MultiMedia,"Consulting,Data Visualization,Digital Media","Data and Analytics,Design,Information Technolo...",Houston,Houston,2007-01-01,operating,http://www.smartmm.com,0.733041,...,0,1,1,130,168,5427,12,12,4773,0
3,000095de-8e2b-82f1-32a7-c222ba3d5682,Ultraprise Loan Technologies,"B2B,E-Commerce",Commerce and Shopping,Hagerstown,Frederick,2001-01-01,operating,0,0.732135,...,0,1,1,50,195,-1,7,-1,-1,0
4,0000d497-c93a-eea3-eeb0-a943dfb4f71e,AutoOffer,"Automotive,Insurance,Trading Platform","Financial Services,Lending and Investments,Tra...",Copenhagen,Copenhagen,2013-01-01,operating,https://www.autooffer.dk/,0.687143,...,0,0,1,48,352,24562,17,24,6666,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320910,ffff25bc-9902-33e8-062e-6d84841d8cf9,Vivoxid,Health Care,Health Care,Turku,Turku,2001-01-01,closed,http://www.vivoxid.com,0.737359,...,0,1,1,313,-1,-1,22,-1,-1,0
320911,ffff3209-5481-022e-cdbf-c38a0a762dc4,BJJ Jiu-Jitsu,Internet,Internet Services,0,0,2012-01-01,operating,http://www.jiujitsutoday.com/,0.000000,...,0,1,1,363,-1,-1,24,-1,-1,0
320912,ffff8343-97a6-3667-67af-ed3789fa9e14,Wantist,"E-Commerce,Gift Card,Shopping","Commerce and Shopping,Financial Services","Portland, Oregon",Portland,2010-11-01,operating,http://www.wantist.com,0.683939,...,1,0,1,204,290,23666,7,16,-1,0
320913,ffffabce-6d4a-b3d1-13c0-4e90cedf5270,ERNIT,"Enterprise Applications,Finance,Mobile Apps","Apps,Financial Services,Mobile,Software",New York City,New York,2015-01-01,operating,http://www.ernit.com/,0.672656,...,0,0,6,234,255,18365,3,16,6280,1


In [28]:
df_final_all.to_csv('dataset_ready2.csv')

